In [40]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import csv
import time

from config import api_key

In [4]:
base_url = 'http://api.sportradar.us/mlb/trial/v6.5/en/games/{}/{}.json?api_key={}'
game_url = base_url.format('2018/PST', 'schedule', api_key)

resp = requests.get(game_url).json()
postseason_games = [x['id'] for x in resp['games']]
postseason_games

['000023f8-e153-4dcd-9ba0-627ecc9f162e',
 '09e05863-db41-4346-b007-c58201655f9a',
 '0a408f5b-3ea6-4500-84d4-d01a56b3f3c2',
 '0af06a7c-55e9-49e2-bf18-ba91e87f112f',
 '0c29784b-96cf-478c-b2b4-5dc59c4dac2f',
 '0dc994a5-3fe9-419d-a6dd-0727c01c38ef',
 '2a747bb0-2213-4db6-8173-14a8a1060c36',
 '2aee13d3-9bbe-4e51-a6a7-22fdb01f6fb3',
 '37d6d617-2b4a-4c81-b257-504bbf6793ed',
 '3919001f-caa4-4d5f-9087-dbf5dfb23542',
 '3caab0f7-253b-4bde-bf45-9e0838fc2118',
 '472ad042-b1e0-4a15-93ee-34297fcfeb2d',
 '4aaf2a8a-cab1-4085-b011-fe555e723193',
 '4ecec54e-d51b-483d-831d-a2c535d6e8fa',
 '560f46cd-679d-4bf6-93bb-74b322d33c17',
 '5b58eaf5-792a-4c9f-9c83-39094d8b566e',
 '5d1f85bd-4f87-47cc-9975-52dedd2302cc',
 '5da8f654-17ad-4664-a043-02a6f3b5e23d',
 '6721b2fd-829c-472a-bb23-026068987859',
 '6b10edd9-8a3f-4e5e-bcc5-e65de6c68cff',
 '6b7e96c5-b7ee-449d-8756-cbb25b2ce636',
 '7a524055-608d-45f4-9219-2e6b3ad47a0b',
 '838f51b1-8fa9-4742-ad1c-db05ef23f9a5',
 '84d375dc-ec1e-4bb5-9260-c018c820dad4',
 '86a0958a-2b1b-

In [5]:
games = list()

i = 0
k = 0

for game in postseason_games:
    try:
        k += 1
        time.sleep(1.1)
        game_url = base_url.format(game, 'pbp', api_key)
        game_resp = requests.get(game_url).json()
        if 'pitch_x' in game_resp['game']['innings'][1]['halfs'][0]['events'][0]['at_bat']['events'][0]['pitcher'].keys():
            games.append(game)
            i += 1
        print(f'{i}/{k}')
    except:
        print(f'{i}/{k}')

1/1
1/2
2/3
3/4
4/5
5/6
6/7
6/8
7/9
7/10
8/11
9/12
10/13
10/14
11/15
12/16
13/17
13/18
14/19
15/20
16/21
17/22
18/23
19/24
20/25
21/26
22/27
23/28
23/29
24/30
24/31
25/32
26/33
26/34
26/35
27/36
28/37
28/38
29/39
30/40
31/41
32/42
33/43


In [6]:
def get_umpire(game_id, api_key):
    umpire_url = base_url.format(game_id, 'summary', api_key)
    # umpire_url = f'http://api.sportradar.us/mlb/trial/v6.5/en/games/{game_id}/summary.json?api_key={api_key}'
    resp = requests.get(umpire_url).json()
    officials = resp['game']['officials']
    for official in officials:
        if official['assignment'] == 'HP':
            return official['full_name']
        

def get_game_by_id(game_id, api_key):
    game_url = base_url.format(game_id, 'pbp', api_key)
    # game_url = f'http://api.sportradar.us/mlb/trial/v6.5/en/games/{game_id}/pbp.json?api_key={api_key}'
    game_resp = requests.get(game_url).json()
    return game_resp

In [7]:
outcome_ids = ['kKL', 'bB', 'aKLAD1', 'aKLAD2', 'aKLAD3', 'aKLAD4', 'oKLT1', 'oKLT2', 'oKLT3', 
               'oKLT4']
csv_header = ['Umpire', 'Game_ID', 'Home/Away', 'Called Ball/Strike', 'Pitch_X', 'Pitch_Y', 
              'Pitcher_Hand', 'Batter_Hand', 'Pitch_Type', 'Pitch_Speed']

with open('Resources/pitch_data.csv', 'w', newline='') as fout:
    csvwriter = csv.writer(fout, delimiter=',')
    csvwriter.writerow(csv_header)
    for game in games:
        time.sleep(1.1)
        game_json = get_game_by_id(game, api_key)
        time.sleep(1.1)
        umpire = get_umpire(game, api_key)
        for inning in game_json['game']['innings'][1:]:
            for half in inning['halfs']:
                t_b = half['half']
                if t_b == 'T':
                    t_b = 'Away'
                else:
                    t_b = 'Home'
                for event in half['events']:
                    try:
                        for pitch in event['at_bat']['events']:
                            if pitch['outcome_id'] in outcome_ids:
                                if pitch['outcome_id'] == 'bB':
                                    call = 'Ball'
                                else:
                                    call = 'Strike'
                                pitch_type = pitch['pitcher']['pitch_type']
                                pitch_speed = pitch['pitcher']['pitch_speed']
                                pitcher_hand = pitch['pitcher']['pitcher_hand']
                                batter_hand = pitch['pitcher']['hitter_hand']
                                pitch_x = pitch['pitcher']['pitch_x']
                                pitch_y = pitch['pitcher']['pitch_y']
                                csvwriter.writerow([umpire, game, t_b, call, pitch_x, pitch_y, 
                                                    pitcher_hand, batter_hand, pitch_type, 
                                                    pitch_speed])
                    except:
                        pass
                            
print('done')

done


In [51]:
pitch_df = pd.read_csv('Resources/pitch_data.csv')
pitch_df.head()

,Umpire,Game_ID,Home/Away,Called Ball/Strike,Pitch_X,Pitch_Y,Pitcher_Hand,Batter_Hand,Pitch_Type,Pitch_Speed
0,James Hoye,000023f8-e153-4dcd-9ba0-627ecc9f162e,Away,Ball,-170,-47,L,R,FA,91.0
1,James Hoye,000023f8-e153-4dcd-9ba0-627ecc9f162e,Away,Strike,82,-21,L,R,FA,89.0
2,James Hoye,000023f8-e153-4dcd-9ba0-627ecc9f162e,Away,Ball,41,152,L,R,FA,90.0
3,James Hoye,000023f8-e153-4dcd-9ba0-627ecc9f162e,Away,Ball,-88,143,L,R,FA,94.0
4,James Hoye,000023f8-e153-4dcd-9ba0-627ecc9f162e,Away,Ball,-200,-34,L,R,FA,93.0


In [ ]:
pitch_dictionary = {'FA': 'Fastball',
                    'SI': 'Sinker',
                    'CT': 'Cutter',
                    'CU': 'Curveball',
                    'SL': 'Slider',
                    'CH': 'Changeup',
                    'KN': 'Knuckleball',
                    'SP': 'Splitter',
                    'SC': 'Screwball',
                    'FO': 'Forkball',
                    'IB': 'Intentional Ball',
                    'PI': 'Pitchout',}

pitch_df['Theoretical_Call'] = ''
pitch_df['Accurate'] = ''
pitch_df['Strike'] = ''

for i, row in pitch_df.iterrows():
    pitch_x = abs(row['Pitch_X'])
    pitch_y = abs(row['Pitch_Y'])
    pitch_type = row['Pitch_Type']
    pitch_df.loc[i, 'Pitch_Type'] = pitch_dictionary.get(pitch_type, 'NA')
    if (pitch_x > 100) or (pitch_y > 100):
        pitch_df.loc[i, 'Theoretical_Call'] = 'Ball'
        pitch_df.loc[i, 'Strike'] = False
    else:
        pitch_df.loc[i, 'Theoretical_Call'] = 'Strike'
        pitch_df.loc[i, 'Strike'] = True
    if pitch_df.loc[i, 'Called Ball/Strike'] == pitch_df.loc[i, 'Theoretical_Call']:
        pitch_df.loc[i, 'Accurate'] = True
    else:
        pitch_df.loc[i, 'Accurate'] = False

pitch_df.head()

In [ ]:
acc_data = pitch_df["Accurate"].value_counts()
incrt_df = pitch_df.loc[pitch_df['Accurate'] == 0]
incrt_data = incrt_df['Called Ball/Strike'].value_counts()

explode = [0.1, 0]
colors = ["lightskyblue", "orange"]
labels = ["Correct Calls", "Incorrect Calls"]
incrt_labels = ["Called Strike", "Called Ball"]

fig = plt.figure()
ax1 = fig.add_axes([0, 0, 0.75, 1.5], aspect=1)
ax1.pie(acc_data, explode=explode, labels=labels, autopct='%0.2f%%', colors=colors)
ax2 = fig.add_axes([1, .0, 0.75, 1.5], aspect=1)
ax2.pie(incrt_data, explode=explode, labels=incrt_labels, autopct='%0.2f%%', colors=colors)
ax1.set_title("Overall Correct vs Incorrect Calls 2018 Post-Season")
ax2.set_title("Incorrect Calls: Balls vs Strikes")

plt.figure(figsize=(20,10))
plt.tight_layout()

fig.savefig("Images/dbl_pie_incorrect_calls.png",bbox_inches = "tight")
plt.show()

In [ ]:
umpire_group = pitch_df.groupby('Umpire')
incrt_umpire_group = incrt_df.groupby('Umpire')

total_calls = umpire_group['Called Ball/Strike'].count()
missed_calls = incrt_umpire_group['Accurate'].count()

ump_calls_df = pd.DataFrame({"Total Calls": total_calls,
                            "Missed Calls": missed_calls,
                            "Missed Call Percentage": missed_calls / total_calls,})

ump_calls_df['Umpire'] = ump_calls_df.index
ump_calls_df = ump_calls_df.sort_values(by=["Missed Call Percentage"], ascending = False)

ump_calls_df.head()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 13
fig_size[1] = 5
plt.rcParams["figure.figsize"] = fig_size

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ump_calls_df.plot(kind='bar', x='Umpire', y='Total Calls', ax=ax1, color='blue', 
                  legend=True).legend(bbox_to_anchor=(1,1))
ump_calls_df['Missed Call Percentage'].plot(ax=ax2, color='red', 
                                            legend=True).legend(bbox_to_anchor= (1, 0.9))

plt.title('Total Pitches Called and Missed Called % By Umpire')
plt.xlabel('Umpire')
ax1.set_ylabel('Total Pitches Called')
ax2.set_ylabel('Missed Call Percentage')

mean_line = pitch_df["Accurate"].mean() * 100
mean_line_round = round(mean_line,4)
incrt_mean = (1 - (mean_line_round/100))
plt.axhline(y=incrt_mean, color = "black", ls = "dashed")
plt.text(24, incrt_mean+0.005, (f'Mean % = {incrt_mean*100:.2f}%'))

vals = ax2.get_yticks()
frmt = ['{:.0f}%'.format(val*100) for val in vals]
ax2.set_yticks(vals)
ax2.set_yticklabels(frmt)

plt.tight_layout()

fig.savefig("Images/calls_by_ump.png")
plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size

strikes = pitch_df.loc[pitch_df["Called Ball/Strike"] == "Strike"]
balls = pitch_df.loc[pitch_df["Called Ball/Strike"] == "Ball"]

x_axis_b = balls["Pitch_X"]
y_axis_b = balls["Pitch_Y"]
x_axis_s = strikes["Pitch_X"]
y_axis_s = strikes["Pitch_Y"]

plt.figure()
plt.scatter(x_axis_b, y_axis_b, marker="o", facecolors="green",edgecolors="black", alpha=0.75)
plt.scatter(x_axis_s, y_axis_s, marker="o", facecolors="red",edgecolors="black", alpha=0.75)
plt.grid()
plt.title("Plot of strikes and balls for the post season 2018")
plt.xlabel("Horizontal strike zone")
plt.ylabel("Vertical strike zone")
plt.ylim(-300, 300)
plt.xlim(-300, 300)

lgnd = plt.legend(["Ball", "Strike"], loc=1, markerscale=.5, title="Pitch Type")
lgnd.legendHandles[0]._sizes = [30]
lgnd.legendHandles[1]._sizes = [30]

x = [-100, -100, 100, 100, -100]
y = [100, -100, -100, 100, 100]
plt.plot(x, y, linewidth = 5)

fig.savefig("Images/all_pitches_scatter.png",bbox_inches = "tight")
plt.show()

In [ ]:
strikes_bad = strikes.loc[strikes['Accurate'] == False]
balls_bad = balls.loc[balls['Accurate'] == False]

x_axis_strikes_bad = strikes_bad["Pitch_X"]
y_axis_strikes_bad = strikes_bad["Pitch_Y"]
x_axis_balls_bad = balls_bad["Pitch_X"]
y_axis_balls_bad = balls_bad["Pitch_Y"]


temp_plot = plt.scatter(x_axis_strikes_bad, y_axis_strikes_bad, marker="o", 
                        facecolors="blue",edgecolors="black", alpha=0.75)
temp_plot = plt.scatter(x_axis_balls_bad, y_axis_balls_bad, marker="o", 
                        facecolors="yellow",edgecolors="black", alpha=0.75)

plt.grid()

plt.title("Plot of missed calls strikes and balls for the post season 2018")
plt.xlabel("Horizontal strike zone")
plt.ylabel("Vertical strike zone")

lgnd= plt.legend(["Called Strike", "Called Ball"], loc=1, markerscale = .5, title="Pitch Type")
lgnd.legendHandles[0]._sizes = [30]
lgnd.legendHandles[1]._sizes = [30]

x = [-100, -100, 100, 100, -100]
y = [100, -100, -100, 100, 100]
plt.plot(x, y, linewidth = 5)

plt.ylim(-300,300)
plt.xlim(-300,300)

plt.savefig("Images/missed_calls_strikes_balls.png", bbox_inches="tight")
plt.show()

In [ ]:
bins = (65,70,75,80,85,90,95,100,105)
group_names = ("65-70", "71-75", "76-80", "81-85", "86-90", "91-95", "95-100", "101+")
pitch_df["Speed Bin"] = pd.cut(pitch_df.loc[:, "Pitch_Speed"], bins, labels=group_names)
incrt_df["Speed Bin"] = pd.cut(pitch_df.loc[:, "Pitch_Speed"], bins, labels=group_names)

total_calls = pitch_df.groupby('Speed Bin')['Called Ball/Strike'].count()
missed_calls = incrt_df.groupby('Speed Bin')['Accurate'].count()

pitch_speed_df = pd. DataFrame({"Total Calls": total_calls,
                                "Incorrect Calls": missed_calls,
                                "Incorrect %": missed_calls / total_calls,})
pitch_speed_df.fillna(0, inplace=True)

pitch_speed_df['Pitch Speed (MPH)'] = pitch_speed_df.index

pitch_speed_df

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 11
fig_size[1] = 6
plt.rcParams["figure.figsize"] = fig_size

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

pitch_speed_df.plot(kind='bar', x='Pitch Speed (MPH)', y='Total Calls', ax=ax1, 
                    color='blue', legend=True).legend(bbox_to_anchor=(1,1))
pitch_speed_df['Incorrect %'].plot(ax=ax2, color='red', 
                                   legend=True).legend(bbox_to_anchor=(1,0.9))

plt.title('Total Pitches Called and Missed Called % by Pitch Speed')
plt.xlabel('Pitch Speed (MPH)')
ax1.set_ylabel('Total Pitches Called')
ax2.set_ylabel('Missed Call Percentage')

incrt_mean = 1 - (mean_line_round/100)
plt.axhline(y=incrt_mean, color = "black", ls = "dashed")
plt.text(6.6, incrt_mean+0.005, (f'Mean = {incrt_mean*100:.2f}%'))

vals = ax2.get_yticks()
ax2.set_yticklabels(['{:,.2%}'.format(x) for x in vals])

plt.tight_layout()

fig.savefig('Images/calls_pitch_speed.png')
plt.show()

In [ ]:
total_calls = pitch_df.groupby('Pitch_Type')['Called Ball/Strike'].count()
missed_calls = incrt_df.groupby('Pitch_Type')['Accurate'].count()

pitch_type_df = pd.DataFrame({"Total Calls": total_calls,
                              "Missed Calls": missed_calls,
                              "Missed Call %": missed_calls / total_calls,})

pitch_type_df['Pitch Type'] = pitch_type_df.index
pitch_type_df = pitch_type_df.sort_values(by=["Missed Call %"], ascending = False)

pitch_type_df

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

pitch_type_df.plot(kind='bar', x='Pitch Type', y='Total Calls', ax=ax1, color='blue', 
                   legend=True).legend(bbox_to_anchor=(1,1))
pitch_type_df['Missed Call %'].plot(ax=ax2, color='red', 
                                    legend=True).legend(bbox_to_anchor=(1,0.95))

plt.title('Total Pitches Called and Missed Called % by Pitch Type')
plt.xlabel('Pitch Type')
ax1.set_ylabel('Total Pitches Called')
ax2.set_ylabel('Missed Call Percentage')

incrt_mean = 1 - (mean_line_round/100)
ax2.axhline(y=incrt_mean, color="black", ls="dashed")
plt.text(5.5, incrt_mean+0.005, (f'Mean % = {incrt_mean*100:.2f}%'))

vals = ax2.get_yticks()
frmt = ['{:.0f}%'.format(val*100) for val in vals]
ax2.set_yticks(vals)
ax2.set_yticklabels(frmt)

plt.tight_layout()

fig.savefig('Images/calls_pitch_type.png')
plt.show()

In [ ]:
home_inc = len(incrt_df.loc[incrt_df["Home/Away"] == "Home"])
correct_df = pitch_df.loc[pitch_df["Accurate"] == True]
home_crt = len(correct_df.loc[correct_df["Home/Away"] == "Home"])
home_crt_percent = home_crt / (home_inc + home_crt) * 100
home_inc_percent = (100 - home_crt_percent)

away_inc = len(incrt_df.loc[incrt_df["Home/Away"] == "Away"])
away_crt = len(correct_df.loc[correct_df["Home/Away"] == "Away"])
away_crt_percent = away_crt / (away_crt + away_inc) * 100
away_inc_percent = (100 - away_crt_percent)

correct_calls = [away_crt_percent, home_crt_percent]
incorrect_calls = [away_inc_percent, home_inc_percent]

mean_line = pitch_df["Accurate"].mean() * 100
mean_line_round = round(mean_line,4)

fig = plt.figure()
N = 2

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 2
plt.rcParams["figure.figsize"] = fig_size

ind = np.arange(N)
p1 = plt.barh(ind, correct_calls)
p2 = plt.barh(ind, incorrect_calls, left = correct_calls)

plt.xlabel("% of Correct Calls")
plt.yticks(ind,("Away", "Home"))
plt.xticks(np.arange(80, 100, 5))
plt.xlim(80,100)
plt.title("Correct and Incorrect Calls for Home and Away Teams")
plt.legend((p1[0], p2[0]), ('Correct Call', 'Incorrect Call'), loc=9, bbox_to_anchor=(1, -0.1))

plt.axvline(x=mean_line_round, color = "yellow", ls = "dashed")
plt.text(91.5, 0.9, (f"Mean = {mean_line_round:.2f}%"))

plt.tight_layout()
fig.savefig("Images/home_away_calls.png")
plt.show()

In [ ]:
total_aways = pitch_df.loc[pitch_df['Home/Away'] == 'Away'].groupby('Umpire')['Pitch_X'].count()
missed_calls = pitch_df.loc[pitch_df['Accurate'] == False]
missed_aways = missed_calls.loc[missed_calls['Home/Away'] == 'Away'].groupby('Umpire')['Accurate'].count()
missed_homes = missed_calls.loc[missed_calls['Home/Away'] == 'Home'].groupby('Umpire')['Accurate'].count()
all_pitches = pitch_df.groupby('Umpire')['Game_ID'].count()

all_data = pd.DataFrame({'Game_ID': all_pitches,
                         'Missed_Away': missed_aways,
                         'Missed_Home': missed_homes,
                         'Total Aways': total_aways,})

all_data['Total Homes'] = all_data['Game_ID'] - all_data['Total Aways']                            
all_data['% Missed Home'] = all_data['Missed_Home'] / all_data['Total Homes'] 
all_data['% Missed Away'] = all_data['Missed_Away'] / all_data['Total Aways'] 
all_data['Home/Away Delta'] = (all_data['% Missed Home'] - all_data['% Missed Away']) * 100

all_data.head()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

all_data = all_data.sort_values('Home/Away Delta')

bar_colors = list()
for number in all_data['Home/Away Delta']:
    if number <= 0:
        bar_colors.append('g')
    else:
        bar_colors.append('b')
        
all_data['Home/Away Delta'].plot(kind='bar', align='center', color=bar_colors)
plt.ylabel('<- Favors Home || Favors Away ->')
plt.title('Favoring Home or Away?')
plt.margins(0.02)
plt.ylim(-15, 15)

plt.savefig('Images/favor_home_or_away.png')
plt.show()

In [ ]:
rh_df = pitch_df.loc[pitch_df["Batter_Hand"] =="R"]
lh_df = pitch_df.loc[pitch_df["Batter_Hand"] =="L"]

rh_data = rh_df["Accurate"].value_counts()
lh_data = lh_df["Accurate"].value_counts()

hand_labels = ["Correct Calls", "Incorrect Calls"]

fig = plt.figure()
ax1 = fig.add_axes([0, 0, 0.75, 1.5], aspect=1)
ax1.pie(rh_data, explode=explode, labels=hand_labels, autopct='%0.2f%%', colors=colors)
ax2 = fig.add_axes([1, .0, 0.75, 1.5], aspect=1)
ax2.pie(lh_data, explode=explode, labels=hand_labels, autopct='%0.2f%%', colors=colors)

ax1.set_title("Accuracy of Calls for Right Handed Batters")
ax2.set_title("Accuracy of Calls for Left Handed Batters")

plt.figure(figsize=(20,10))
plt.tight_layout()

fig.savefig("Images/dbl_pie_right_left.png", bbox_inches="tight")
plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 4
plt.rcParams["figure.figsize"] = fig_size

right_hand_x = pitch_df[pitch_df['Batter_Hand'] == 'R']['Pitch_X']
right_hand_y = pitch_df[pitch_df['Batter_Hand'] == 'R']['Pitch_Y']
left_hand_x = pitch_df[pitch_df['Batter_Hand'] == 'L']['Pitch_X']
left_hand_y = pitch_df[pitch_df['Batter_Hand'] == 'L']['Pitch_Y']

fig = plt.figure()
axes = fig.add_subplot(121)
plt.boxplot([right_hand_y, left_hand_y])
axes.set_xticklabels(['Right', 'Left'])
plt.ylabel('Vertical Pitch Location')
plt.xlabel('Batter Hand')
axes = fig.add_subplot(122)
plt.boxplot([right_hand_x, left_hand_x], vert=False)
axes.set_yticklabels(['Right', 'Left'])
plt.ylabel('Batter Hand')
plt.xlabel('Horizontal Pitch Location')
plt.tight_layout()
plt.savefig('Images/pitch_location_boxplots.png')